# 1 신경망 복습

* 1.1 수학과 파이썬 복습 (pass)
* 1.2 신경망의 구조 (pass)
* 1.3 신경망의 학습
* 1.4 신경망으로 문제를 풀다
* 1.5 계산 고속화
* 1.6 정리

## 1.3 신경망의 학습

* 학습되지 않은 신경망은 '좋은 추론'을 해낼 수 없다.
* 신경망 학습은 최적의 매개변수 값을 찾는 작업이다. 

### 1.3.1 손실 함수

* 신경망 학습에서 사용하는 지표는 '손실'이다.
* '손실'은 학습 데이터와 신경망이 예측한 결과를 비교하여 얼마나 '나쁜가'를 나타낸다.
* 신경망의 손실은 '손실 함수'로 구한다.
* 다중 클래스 분류 신경망에서는 '교차 엔트로피 오차'를 사용한다.
* 교차 엔트로피 오차는 정답 레이블과 신경망의 출력 간의 오차를 나타낸다.

**소프트맥스 함수**
$$
y_{k} = \frac{exp(s_{k})}{\sum_{i=1}^{n}exp(s_{i})}
$$

* 출력이 총$n$개일 때, $k$번째 출력 $y_{k}$를 구하는 식이다.
* $y_{k}$는 $k$번째 클랙스에 해당하는 소프트맥스 함수의 출력이다.
* 소프트맥스 함수의 분자는 점수$s_{k}$의 지수 함수이고, 분모는 모든 클래스의 점수의 지수 함수의 합이다.
* 소프트맥스 함수의 출력은 0에서 1사이의 실수이며, 출력의 총합은 1이다.

**교차 엔트로피 오차**
$$
L = -\sum_{k}t_{k}log(y_{k})
$$

* $t_{k}$는 $k$번째 클래스에 해당하는 정답 레이블이다.
* log는 네이피어 상수 $e$를 밑으로 하는 자연로그이다.
* 정답 레이블은 원-핫 인코딩으로 표현한다.

**미니배치를 고려한 교차 엔트로피 오차**
$$
L = -\frac{1}{N}\sum_{n}\sum_{k}t_{nk}log(y_{nk})
$$

### 1.3.2 미분과 기울기

1.3.3 연쇄 법칙

* 신경망의 기울기는 '오차역전파법'으로 구한다.

1.3.4 계산 그래프

* 계산 그래프는 계산 과정을 그래프로 나타낸 것이다.

### 1.3.5 기울기 도출과 역전파 구현

**Sigmoid 계층**

시그모이드의 함수는 $$y = \frac{1}{1+exp(-x)}$$
시그모이드의 미분은 $$\frac{dy}{dx} = y(1-y)$$

In [1]:
import numpy as np

class Sigmoid:
  def __init__(self):
    self.params, self.grads = [], []
    self.out = None
    
  def forward(self, x):
    out = 1/(1+np.exp(-x))
    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

In [2]:
# test Sigmod class

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
sigmoid = Sigmoid()
print(sigmoid.forward(x))
sigmoid.out = sigmoid.forward(x)
print(sigmoid.backward(1))
print(sigmoid.backward(1).shape)
print(sigmoid.backward(1).dtype)


[[0.73105858 0.37754067]
 [0.11920292 0.95257413]]
[[0.19661193 0.23500371]
 [0.10499359 0.04517666]]
(2, 2)
float64


**Affine 계층**

Affine계층의 순전파는 $y=xW+b$이다. `np.matmul(x, W) + b`로 구현할 수 있다.

Affine계층의 역전파는 $dy = dxW^{T}$, $dW = x^{T}dy$, $db = \sum_{i}dy_{i}$이다.

In [5]:
class Affine:
  def __init__(self, W, b):
    self.params = [W, b]
    self.grads = [np.zeros_like(W), np.zeros_like(b)]
    self.x = None
    
  def forward(self, x):
    W, b = self.params
    out = np.matmul(x, W) + b
    self.x = x
    return out
  
  def backward(self, dout):
    W, b = self.params
    dx = np.matmul(dout, W.T)
    dW = np.matmul(self.x.T, dout)
    db = np.sum(dout, axis=0)
    
    self.grads[0][...] = dW
    self.grads[1][...] = db
    return dx

In [6]:
# test Affine class
W = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([7, 8, 9])
x = np.array([[1, 2]])
affine = Affine(W, b)
print(affine.forward(x))
print(affine.backward(1))
print(affine.backward(1).shape)
print(affine.backward(1).dtype)

[[16 20 24]]


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)